In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd gdrive/MyDrive/Colab\ Notebooks/RL/vol-surface-vae
except:
    pass

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from vae.datasets import SABRDataset, SABRExFeatsDataset
from vae.cvae_with_mem import CVAEMem
from vae.utils import *

In [ ]:
set_seeds(0)
torch.set_default_dtype(torch.float64)

In [ ]:
seq_len = 5
ctx_len = seq_len - 1
num_epochs = 50

In [ ]:
data = np.load("data/sabr_surface_with_ret.npz")
vol_surf_data = data["surface"]
ret_data = data["ret"]
price_data = data["price"]
train_ret = DataLoader(SABRExFeatsDataset(vol_surf_data[:5000], ret_data[:5000], seq_len), shuffle=True, batch_size=64)
valid_ret = DataLoader(SABRExFeatsDataset(vol_surf_data[5000:6000], ret_data[5000:6000], seq_len), shuffle=True, batch_size=16)
test_ret = DataLoader(SABRExFeatsDataset(vol_surf_data[6000:7000], ret_data[6000:7000], seq_len), shuffle=True, batch_size=16)

train_price = DataLoader(SABRExFeatsDataset(vol_surf_data[:5000], price_data[:5000], seq_len), shuffle=True, batch_size=64)
valid_price = DataLoader(SABRExFeatsDataset(vol_surf_data[5000:6000], price_data[5000:6000], seq_len), shuffle=True, batch_size=16)
test_price = DataLoader(SABRExFeatsDataset(vol_surf_data[6000:7000], price_data[6000:7000], seq_len), shuffle=True, batch_size=16)

train_simple = DataLoader(SABRDataset(vol_surf_data[:5000], seq_len), shuffle=True, batch_size=64)
valid_simple = DataLoader(SABRDataset(vol_surf_data[5000:6000], seq_len), shuffle=True, batch_size=16)
test_simple = DataLoader(SABRDataset(vol_surf_data[6000:7000], seq_len), shuffle=True, batch_size=16)

# Conv version

## ret as ex feat

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 25,
    "device": "cuda",
    "kl_weight": 1,
    "re_feat_weight": 1,
    "surface_hidden": [2, 4, 5],
    "ex_feats_dim": 1,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [2, 4, 5], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": False,
}
model = CVAEMem(config)
print(model)

In [ ]:
train(model, train_ret, valid_ret, epochs=num_epochs, lr=1e-05, model_dir="test/ret", file_name="conv2d_mem_sabr_ret.pt")

In [ ]:
test(model, valid_ret, test_ret, "test/ret/conv2d_mem_sabr_ret.pt")

In [ ]:
surf, ex_feat = model.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[8000][0:ctx_len]), 
                                    "ex_feats": torch.from_numpy(ret_data[8000][0:ctx_len]).unsqueeze(-1)})
surf = surf.detach().cpu().numpy().reshape((5,5))
ex_feat = ex_feat.detach().cpu().numpy().reshape((1,))[0]
print(f"Real return: {ret_data[8000][ctx_len]}, predicted: {ex_feat}")
plot_surface_separate(vol_surf_data[8000][ctx_len], surf)

In [ ]:
model_data = torch.load("test/ret/conv2d_mem_sabr_ret.pt")
sim = generate_surface_path(vol_surf_data, ret_data, model_data, 8000)
plot_surface_time_series(sim)

## price as ex feats

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 25,
    "device": "cuda",
    "kl_weight": 1,
    "re_feat_weight": 1,
    "surface_hidden": [2, 4, 5],
    "ex_feats_dim": 1,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [2, 4, 5], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": False,
}
model_price = CVAEMem(config)
print(model_price)

In [ ]:
train(model_price, train_price, valid_price, epochs=num_epochs, lr=1e-05, model_dir="test/price", file_name="conv2d_mem_sabr_price.pt")

In [ ]:
test(model_price, valid_price, valid_price, "test/price/conv2d_mem_sabr_price.pt")

In [ ]:
surf, ex_feat = model_price.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[8000][0:ctx_len]), 
                                    "ex_feats": torch.from_numpy(price_data[8000][0:ctx_len]).unsqueeze(-1)})
surf = surf.detach().cpu().numpy().reshape((5,5))
ex_feat = ex_feat.detach().cpu().numpy().reshape((1,))[0]
print(f"Real price: {price_data[8000][ctx_len]}, predicted: {ex_feat}")
plot_surface_separate(vol_surf_data[8000][ctx_len], surf)

In [ ]:
model_data = torch.load("test/price/conv2d_mem_sabr_price.pt")
sim = generate_surface_path(vol_surf_data, price_data, model_data, 8000)
plot_surface_time_series(sim)

## No extra features

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 25,
    "device": "cuda",
    "kl_weight": 1.0,
    "re_feat_weight": 1,
    "surface_hidden": [2, 4, 5],
    "ex_feats_dim": 0,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [2, 4, 5], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": False,
}
model_simple = CVAEMem(config)
print(model_simple)

In [ ]:
train(model_simple, train_simple, valid_simple, epochs=num_epochs, lr=1e-05, model_dir="test/simple", file_name="conv2d_mem_sabr_no_ex_feats.pt")

In [ ]:
test(model_simple, valid_simple, test_simple, "test/simple/conv2d_mem_sabr_no_ex_feats.pt")

In [ ]:
surf = model_simple.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[8000][0:ctx_len])})
surf = surf.detach().cpu().numpy().reshape((5,5))
plot_surface_separate(vol_surf_data[8000][ctx_len], surf)

In [ ]:
model_data = torch.load("test/simple/conv2d_mem_sabr_no_ex_feats.pt")
sim = generate_surface_path(vol_surf_data, price_data, model_data, 8000)
plot_surface_time_series(sim)

# Dense

## ret as ex feats

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 25,
    "device": "cuda",
    "kl_weight": 0.8,
    "re_feat_weight": 1,
    "surface_hidden": [25, 50, 50],
    "ex_feats_dim": 1,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [25, 50, 50], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": True,
}
model2 = CVAEMem(config)
print(model2)

In [ ]:
train(model2, train_ret, valid_ret, epochs=num_epochs, lr=1e-05, model_dir="test/ret", file_name="dense_mem_sabr_ret.pt")

In [ ]:
test(model2, valid_ret, test_ret, "test/ret/dense_mem_sabr_ret.pt")

In [ ]:
surf, ex_feat = model2.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[8000][0:ctx_len]), 
                                    "ex_feats": torch.from_numpy(ret_data[8000][0:ctx_len]).unsqueeze(-1)})
surf = surf.detach().cpu().numpy().reshape((5,5))
ex_feat = ex_feat.detach().cpu().numpy().reshape((1,))[0]
print(f"Real return: {ret_data[8000][ctx_len]}, predicted: {ex_feat}")
plot_surface_separate(vol_surf_data[8000][ctx_len], surf)

In [ ]:
model_data = torch.load("test/ret/dense_mem_sabr_ret.pt")
sim = generate_surface_path(vol_surf_data, ret_data, model_data, 8000)
plot_surface_time_series(sim)

## price as ex feats

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 25,
    "device": "cuda",
    "kl_weight": 1,
    "re_feat_weight": 1,
    "surface_hidden": [25, 50, 50],
    "ex_feats_dim": 1,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [25, 50, 50], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": True,
}
model_price2 = CVAEMem(config)
print(model_price2)

In [ ]:
train(model_price2, train_price, valid_price, epochs=num_epochs, lr=1e-05, model_dir="test/price", file_name="dense_mem_sabr_price.pt")

In [ ]:
test(model_price2, valid_price, test_price, "test/price/dense_mem_sabr_price.pt")

In [ ]:
surf, ex_feat = model_price2.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[8000][0:ctx_len]), 
                                    "ex_feats": torch.from_numpy(price_data[8000][0:ctx_len]).unsqueeze(-1)})
surf = surf.detach().cpu().numpy().reshape((5,5))
ex_feat = ex_feat.detach().cpu().numpy().reshape((1,))[0]
print(f"Real price: {price_data[8000][ctx_len]}, predicted: {ex_feat}")
plot_surface_separate(vol_surf_data[8000][ctx_len], surf)

In [ ]:
model_data = torch.load("test/price/dense_mem_sabr_price.pt")
sim = generate_surface_path(vol_surf_data, price_data, model_data, 8000)
plot_surface_time_series(sim)

## No ex feats

In [ ]:
config = {
    "seq_len": seq_len, 
    "feat_dim": (5, 5),
    "latent_dim": 25,
    "device": "cuda",
    "kl_weight": 1,
    "re_feat_weight": 1,
    "surface_hidden": [25, 50, 50],
    "ex_feats_dim": 0,
    "ex_feats_hidden": None,
    "mem_type": "lstm",
    "mem_hidden": 75,
    "mem_layers": 2,
    "mem_dropout": 0.2,
    "ctx_len": ctx_len, 
    "ctx_surface_hidden": [25, 50, 50], 
    "ctx_ex_feats_hidden": None,
    "interaction_layers": None,
    "use_dense_surface": True,
}
model_simple2 = CVAEMem(config)
print(model_simple2)

In [ ]:
train(model_simple2, train_simple, valid_simple, epochs=num_epochs, lr=1e-05, model_dir="test/simple", file_name="dense_mem_sabr_no_ex_feats.pt")

In [ ]:
test(model_simple2, valid_simple, test_simple, "test/simple/dense_mem_sabr_no_ex_feats.pt")

In [ ]:
surf = model_simple2.get_surface_given_conditions({"surface": torch.from_numpy(vol_surf_data[8000][0:ctx_len])})
surf = surf.detach().cpu().numpy().reshape((5,5))
plot_surface_separate(vol_surf_data[8000][ctx_len], surf)

In [ ]:
model_data = torch.load("test/simple/dense_mem_sabr_no_ex_feats.pt")
sim = generate_surface_path(vol_surf_data, price_data, model_data, 8000)
plot_surface_time_series(sim)